In [1]:
#..,,,;;:! Rise FrankenKernel RISE !:;;,,,..

import numpy as np
import pandas as pd
from sklearn import *
import xgboost as xgb
from collections import Counter
import re

train = pd.read_csv('./input/train.csv').fillna('')
test = pd.read_csv('./input/test.csv').fillna('')

lbl = preprocessing.LabelEncoder()
train['author'] = lbl.fit_transform(train['author'])

#Lets give Franky-Kernel some features
do_the_do_da_dict = {}
for author in train['author'].unique():
    do_the_do_da_dict[author] = re.sub("[^a-zA-Z ]"," ",' '.join(train[train['author']==author]['text'].values))
    do_the_do_da_dict[author] = Counter(do_the_do_da_dict[author].lower().split(' '))
#print(do_the_do_da_dict)

d_unique = {}
for author1 in train['author'].unique():
    words = [w for w in do_the_do_da_dict[author1]]
    for author2 in train['author'].unique():
        if author1 != author2:
            words = [w for w in words if w not in do_the_do_da_dict[author2] and len(w)>2]
    words = [str(j[1]) for j in sorted([[do_the_do_da_dict[author1][w],w] for w in words], reverse=True)] #[:100]
    d_unique[author1] = words
    print(author1, len(d_unique[author1]), d_unique[author1])

tfidf = feature_extraction.text.CountVectorizer(ngram_range=(1, 4), stop_words='english')

params = {
    'eta': 1.0,
    'max_depth': 15,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'num_class': len(train['author'].unique()),
    'seed': 0,
    'silent': True
}

x1 = tfidf.fit_transform(train['text']).tocsc()
y1 = train['author'].values
model = xgb.train(params, xgb.DMatrix(x1, y1), 200,  [(xgb.DMatrix(x1, y1), 'train')], verbose_eval=20, early_stopping_rounds=20)

def unique_word_prob(author, s):
    s = re.sub("[^a-zA-Z ]"," ", s).lower().split(' ')
    prob = 0.0
    for w in d_unique[author]:
        if w in s:
            prob += 1
    return prob / len(d_unique[author])

pred = model.predict(xgb.DMatrix(tfidf.transform(test['text']).tocsc()), ntree_limit=model.best_ntree_limit)
pred = pd.DataFrame(pred, columns=lbl.transform(lbl.classes_))
for c in lbl.transform(lbl.classes_):
    pred[c] = pred[c] * 0.8 + test['text'].map(lambda x: unique_word_prob(c,x)) * 0.2

pred.columns = lbl.classes_
pred['id'] = test['id']
pred.to_csv('submission.csv', index=False)

/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be re

0 5627 ['dupin', 'marie', 'jupiter', 'monsieur', 'maelzel', 'ellison', 'color', 'bug', 'prefect', 'legrand', 'lalande', 'dat', 'compartment', 'diddler', 'lollipop', 'wyatt', 'termed', 'etoile', 'valdemar', 'pompey', 'massa', 'diameter', 'velocity', 'talbot', 'rog', 'endeavored', 'diddle', 'oppodeldoc', 'honor', 'beauvais', 'wilson', 'labor', 'invention', 'des', 'mille', 'espanaye', 'endeavor', 'ebony', 'crab', 'tea', 'scarab', 'pounds', 'presume', 'moskoe', 'metzengerstein', 'mademoiselle', 'ligeia', 'kempelen', 'dollars', 'yxu', 'thingum', 'str', 'puff', 'premises', 'colored', 'beetle', 'morella', 'mesmeric', 'goosetherumfoodle', 'pundit', 'ourang', 'outang', 'mon', 'moissart', 'maillard', 'eustache', 'eugenie', 'berenice', 'zenobia', 'vondervotteimittiss', 'variation', 'satisfactory', 'rowdy', 'roule', 'psyche', 'president', 'parisian', 'neighborhood', 'kate', 'headlong', 'francs', 'fortunato', 'elopement', 'eleonora', 'duc', 'dow', 'deposes', 'demeanor', 'deluc', 'coincidences', 'bl

[0]	train-mlogloss:0.954379
Will train until train-mlogloss hasn't improved in 20 rounds.
[20]	train-mlogloss:0.549591
[40]	train-mlogloss:0.444382
[60]	train-mlogloss:0.376971
[80]	train-mlogloss:0.331454
[100]	train-mlogloss:0.29749
[120]	train-mlogloss:0.270408
[140]	train-mlogloss:0.247501
[160]	train-mlogloss:0.228749
[180]	train-mlogloss:0.212932
